1 - bibliotecas

In [1]:
import arxiv
import requests
import os
import re
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import torch
import faiss
import numpy as np
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from sklearn.metrics.pairwise import cosine_similarity

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2 - download dos artigos

In [ ]:
os.makedirs("pdfs", exist_ok=True)

# 2. Configura o cliente e a busca
client = arxiv.Client()
search = arxiv.Search(
    query="cat:astro-ph",
    max_results=1000,
    sort_by=arxiv.SortCriterion.Relevance
)

print("Iniciando busca e download...")

# 3. Processa os resultados e faz o download
for i, result in enumerate(client.results(search)):
    pdf_url = result.pdf_url
    # Limpa o título para evitar caracteres inválidos no nome do arquivo
    safe_title = "".join(c for c in result.title if c.isalnum() or c in (' ', '_')).rstrip()
    filename = f"pdfs/paper_{i}_{safe_title[:50]}.pdf"
    
    print(f"Baixando: {result.title}...")
    
    try:
        r = requests.get(pdf_url)
        r.raise_for_status() # Verifica se o download deu certo
        
        with open(filename, "wb") as f:
            f.write(r.content)
            
    except Exception as e:
        print(f"Erro ao baixar o artigo {i}: {e}")

print("\nProcesso finalizado!")

3 - extração de textos dos PDFs

In [2]:
texts = []
path = "pdfs"

# Função simples para limpar ruídos comuns de PDFs acadêmicos
def clean_academic_text(text):
    # Remove excesso de espaços e quebras de linha estranhas
    text = re.sub(r'\s+', ' ', text)
    # Remove referências comuns de rodapé ou URLs (opcional, mas ajuda o GPT-2)
    text = re.sub(r'http\S+', '', text)
    return text.strip()

if os.path.exists(path):
    # Lista e filtra apenas arquivos PDF
    files = sorted([f for f in os.listdir(path) if f.endswith(".pdf")])
    
    # Se você quiser testar com os 1000, garantimos que ele pegue todos ou o limite disponível
    print(f"Iniciando a leitura de {len(files)} arquivos...")

    for i, file in enumerate(files):
        try:
            reader = PdfReader(os.path.join(path, file))
            full_text = ""
            
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    full_text += page_text + " "
            
            # Aplicamos a limpeza antes de salvar
            cleaned_text = clean_academic_text(full_text)
            
            if len(cleaned_text) > 100: # Ignora arquivos quase vazios ou corrompidos
                texts.append(cleaned_text)
            
            # Feedback a cada 50 arquivos para não inundar o console do Jupyter
            if (i + 1) % 50 == 0:
                print(f"Progresso: {i + 1}/{len(files)} arquivos processados.")
            
        except Exception as e:
            print(f"Erro no arquivo {file}: {e}")
else:
    print("A pasta 'pdfs' não foi encontrada.")

print(f"\nTotal de textos extraídos com sucesso: {len(texts)}")

Iniciando a leitura de 997 arquivos...
Progresso: 50/997 arquivos processados.
Progresso: 100/997 arquivos processados.
Progresso: 150/997 arquivos processados.
Progresso: 200/997 arquivos processados.
Progresso: 250/997 arquivos processados.
Progresso: 300/997 arquivos processados.
Progresso: 350/997 arquivos processados.
Progresso: 400/997 arquivos processados.


Illegal character in Name Object (b'/FPEFAA+Times New Roman \xcf\xee\xeb\xf3\xe6\xe8\xf0\xed\xfb\xe9')
Illegal character in Name Object (b'/FPEFAA+Times New Roman \xcf\xee\xeb\xf3\xe6\xe8\xf0\xed\xfb\xe9')
Illegal character in Name Object (b'/FPEFAF+Times New Roman \xcf\xee\xeb\xf3\xe6\xe8\xf0\xed\xfb\xe9 \xca\xf3\xf0\xf1\xe8\xe2')
Illegal character in Name Object (b'/FPEFAF+Times New Roman \xcf\xee\xeb\xf3\xe6\xe8\xf0\xed\xfb\xe9 \xca\xf3\xf0\xf1\xe8\xe2')


Progresso: 450/997 arquivos processados.
Progresso: 500/997 arquivos processados.
Progresso: 550/997 arquivos processados.
Progresso: 600/997 arquivos processados.
Progresso: 650/997 arquivos processados.
Progresso: 700/997 arquivos processados.
Progresso: 750/997 arquivos processados.
Progresso: 800/997 arquivos processados.
Progresso: 850/997 arquivos processados.
Progresso: 900/997 arquivos processados.
Progresso: 950/997 arquivos processados.

Total de textos extraídos com sucesso: 997


4 - chunckerização

In [3]:
# Ajustamos o tamanho para 512, que é o limite padrão do BGE-Large
def chunk_text_with_overlap(text, size=500, overlap=50):
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), size - overlap):
        # Cria o bloco com o tamanho definido
        chunk = " ".join(words[i : i + size])
        chunks.append(chunk)
        
        # Para economizar memória com 1000 artigos, paramos se o bloco for muito curto
        if i + size >= len(words):
            break
            
    return chunks

# Criamos as listas para armazenar os blocos e o mapeamento de qual arquivo eles vieram
all_chunks = []
metadata = []

print(f"Iniciando a divisão de {len(texts)} documentos...")

for i, text in enumerate(texts):
    # Geramos os blocos para cada texto
    doc_chunks = chunk_text_with_overlap(text, size=512, overlap=50)
    
    for chunk in doc_chunks:
        all_chunks.append(chunk)
        # Guardamos o índice do documento original para saber a fonte depois
        metadata.append({"doc_index": i})

print(f"Total de documentos: {len(texts)}")
print(f"Total de chunks: {len(all_chunks)}")
print(f"Exemplo do primeiro chunk (100 caracteres): {all_chunks[0][:100]}...")

Iniciando a divisão de 997 documentos...
Total de documentos: 997
Total de chunks: 15604
Exemplo do primeiro chunk (100 caracteres): arXiv:0808.3195v1 [astro-ph] 23 Aug 2008Version October 24, 2018 Oblique Ion Two-Stream Instability ...


5 - embbending

In [4]:
# 1. Carrega o modelo BGE-Large (Aproximadamente 1.3GB)
# Este modelo é top-tier em rankings de busca semântica
print("Carregando o modelo BGE-Large... Isso pode levar um minuto.")
model = SentenceTransformer("BAAI/bge-large-en-v1.5")

# 2. Configuração de Hardware
# Se você tiver uma GPU NVIDIA, o código a usará automaticamente para acelerar os 1000 artigos
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Usando dispositivo: {device}")

# 3. Transforma os chunks em vetores numéricos
# Usamos 'all_chunks' que já possui o tratamento de sobreposição (overlap)
print(f"Iniciando a codificação de {len(all_chunks)} blocos. Aguarde...")
embeddings = model.encode(
    all_chunks, 
    show_progress_bar=True, 
    batch_size=32, # Ajuste o batch_size se houver erro de memória (OOM)
    convert_to_numpy=True
)

print(f"\nFormato da matriz de embeddings: {embeddings.shape}")
# O shape será (número_de_chunks, 1024)
# Note que a dimensão subiu de 384 para 1024, muito mais informação por vetor!

Carregando o modelo BGE-Large... Isso pode levar um minuto.
Usando dispositivo: cuda
Iniciando a codificação de 15604 blocos. Aguarde...


Batches:   0%|          | 0/488 [00:00<?, ?it/s]


Formato da matriz de embeddings: (15604, 1024)


6 - banco vetorial FAISS

In [5]:
# 1. Obtém a dimensão dos vetores dinamicamente
# Agora será 1024 (proveniente do BGE-Large)
dim = embeddings.shape[1]

# 2. Cria o índice usando distância Euclidiana (L2)
# O IndexFlatL2 é o padrão ouro para precisão absoluta (exaustivo)
index = faiss.IndexFlatL2(dim)

# 3. Conversão e Adição dos dados
# Certificamos que os dados estão em float32 para otimização do FAISS
embeddings_f32 = np.array(embeddings).astype("float32")

# Adiciona ao índice
index.add(embeddings_f32)

print(f"✅ Índice FAISS configurado para {dim} dimensões.")
print(f"📊 Total de blocos científicos indexados: {index.ntotal}")

# Dica: Verificando se o índice está treinado (FlatL2 não precisa de treino, mas é bom checar)
if index.is_trained:
    print("🚀 O índice está pronto para buscas de alta precisão.")

✅ Índice FAISS configurado para 1024 dimensões.
📊 Total de blocos científicos indexados: 15604
🚀 O índice está pronto para buscas de alta precisão.


7 - recuperação

In [7]:
def retrieve(query, k=5):
    # 1. Transforma a pergunta usando o BGE-Large
    # Importante: O BGE recomenda adicionar uma instrução curta na query para melhor busca
    instruction = "Represent this sentence for searching relevant passages: "
    q_emb = model.encode([instruction + query]).astype("float32")
    
    # 2. Busca no FAISS
    # Aumentamos a precisão com a busca vetorial de 1024 dimensões
    D, I = index.search(q_emb, k)
    
    # 3. Retorna os textos e a origem (metadata)
    retrieved_chunks = []
    for idx in I[0]:
        # Recuperamos o texto do chunk
        content = all_chunks[idx]
        # Recuperamos o índice do PDF original (metadata que salvamos na etapa 2)
        source_id = metadata[idx]['doc_index']
        retrieved_chunks.append({"text": content, "source": source_id})
        
    return retrieved_chunks

# --- Teste de Recuperação ---
pergunta = "What are the main observations of gravitational waves in neutron stars?"
contextos = retrieve(pergunta, k=5)

print(f"🔎 Pergunta: {pergunta}")
print(f"📚 {len(contextos)} trechos encontrados nos 1000 artigos.\n")

for i, res in enumerate(contextos):
    print(f"📍 [Trecho {i+1}] - Originário do PDF índice: {res['source']}")
    print(f"📄 Conteúdo: {res['text'][:400]}...") 
    print("-" * 50)

🔎 Pergunta: What are the main observations of gravitational waves in neutron stars?
📚 5 trechos encontrados nos 1000 artigos.

📍 [Trecho 1] - Originário do PDF índice: 734
📄 Conteúdo: and crust. However, these eﬀects are know n to be very strong [89], so that the inequality (290) should be taken with a grain of salt. 12.5 Gravitational wave asteroseismology The development of gravitational wave detectors like LIGO [ 66], VIRGO [207], TAMA300 [300] and GEO600 [298] is opening up a new window of astronomical obser vations. With a central density on the order of ∼1015g cm−3, neutr...
--------------------------------------------------
📍 [Trecho 2] - Originário do PDF índice: 529
📄 Conteúdo: arXiv:0808.4002v3 [gr-qc] 11 Mar 2009Gravitational-Wave Extraction from Neutron-Star Oscilla tions: comparing linear and nonlinear techniques. Luca Baiotti,1,2Sebastiano Bernuzzi,3Giovanni Corvino,2,3Roberto De Pietri,3and Alessandro Nagar4,5,6 1Graduate School of Arts and Sciences, University of Tokyo,

8 - minstral

In [135]:
# 1. Liberar Memória do Passo Anterior
# Movemos o modelo de embedding para a CPU para abrir espaço na GPU
if 'model' in globals():
    model.to("cpu")
    
torch.cuda.empty_cache()
gc.collect()

# 2. Configuração de Quantização 4-bit (NF4)
# Essencial para comprimir o modelo de 15GB para ~4.5GB
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16 
)

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

print("Carregando Mistral-7B com otimização para 6GB VRAM...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",
    low_cpu_mem_usage=True,
    # Limita o uso para sobrar VRAM para o sistema operacional
    max_memory={0: "4.8GiB"} 
)

Carregando Mistral-7B com otimização para 6GB VRAM...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

9 - função de prompt

In [136]:
def ask(query):
    # 1. Recuperação
    results = retrieve(query, k=4)
    
    # 2. Organização do contexto (Melhorado para indicar fontes ao modelo)
    context_text = ""
    for i, res in enumerate(results):
        context_text += f"[Reference {i+1}]: {res['text']}\n\n"

    # 3. Prompt Refinado (Adicionado restrições de precisão e densidade)
    # Note que pedimos explicitamente por termos técnicos e relações de escala.
    prompt = (
        f"<s>[INST] SYSTEM: You are a rigorous scientific validator. Use ONLY the provided SOURCES to answer. "
        f"If the specific details (equations, constants, or data) are not in the SOURCES, "
        f"say 'Information not available in dataset'. DO NOT use general knowledge about physics. "
        f"Be precise about Energy Conditions and Wormhole stability as described in the text.\n\n"
        f"SOURCES:\n{context_text[:2000]}\n\n"
        f"QUESTION:\n{query} [/INST]\n"
        f"Scientific Analysis based on Sources:"
    )

    # 4. Tokenização e Geração (Hiperparâmetros equilibrados)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    input_length = inputs.input_ids.shape[1]

    output_tokens = model_llm.generate(
        **inputs,
        max_new_tokens=512,
        # Reduzido de 0.4 para 0.1: Para física, queremos o token mais provável (precisão).
        temperature=0.1, 
        do_sample=True, # Mantém a geração determinística
        # Reduzido de 1.4 para 1.1: Evita termos inventados e preserva o vocabulário técnico.
        repetition_penalty=1.1, 
        top_p=0.9, 
        pad_token_id=tokenizer.eos_token_id
    )

    # 5. Retorno Limpo
    generated_tokens = output_tokens[0][input_length:]
    clean_answer = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    return clean_answer, results

10 - teste final

In [141]:
# 1. Definição da pergunta
pergunta = "how the wormhole"

print(f"🚀 [1/2] Iniciando busca semântica...")

# --- MANIPULAÇÃO DE MEMÓRIA (VITAL PARA 6GB) ---
# Movemos o modelo BGE para a CPU para liberar ~1.5GB para o Mistral
if 'model' in globals():
    model.to("cpu")

torch.cuda.empty_cache()
gc.collect()
# -----------------------------------------------

# 2. Execução da busca e geração
# A função ask chamará o retrieve (que usará o model na CPU, o que é rápido para 1 query)
# E usará o model_llm que já está na GPU
try:
    print(f"🧠 [2/2] Consultando Mistral-7B (4-bit) na GPU...")
    resposta, results = ask(pergunta)

    print("\n" + "═"*60)
    print(f"🌌 RESPOSTA CIENTÍFICA:")
    print("-" * 60)
    print(resposta)
    print("═"*60)

    print("\n📚 FONTES DE APOIO (TOP 3):")
    for i, res in enumerate(results[:3]):
        print(f"  [{i+1}] PDF ID: {res['source']} | Trecho: \"{res['text'][:150]}...\"")

except torch.cuda.OutOfMemoryError:
    print("\n❌ ERRO DE MEMÓRIA: A GPU esgotou. Tentando limpar e reduzir o contexto...")
    torch.cuda.empty_cache()
    # Tente rodar novamente diminuindo o k na função retrieve para 2 ou 3.

except Exception as e:
    print(f"\n❌ Ocorreu um erro: {e}")

🚀 [1/2] Iniciando busca semântica...
🧠 [2/2] Consultando Mistral-7B (4-bit) na GPU...

════════════════════════════════════════════════════════════
🌌 RESPOSTA CIENTÍFICA:
------------------------------------------------------------
From the provided sources, we can see that a wormhole solution is presented in equation (3.57) for degree (θ) equal to 9, which gives us the wormhole solution:

ds² = r(r+r₀)²dt² + 3r₀r⁴ dr² + (r+r₀)⁴ r²dθ²

This solution describes a wormhole spacetime where the critical points lie on the Y-axis. However, it's important to note that this wormhole solution is specific to the given degree value (θ = 9).

Regarding energy conditions and wormhole stability, the sources do not provide explicit information about these aspects for the wormhole solution. They focus more on discussing the behavior of different solutions in the phase plane and their asymptotic behavior.

For a wormhole to be physically viable, it should satisfy the null energy condition (NEC) and the 

11 - metricas de desempenho

11.1 - indice de fidelidade semantica

A métrica de Fidelidade Semântica serve para medir o grau de honestidade intelectual do seu assistente em relação aos documentos fornecidos, funcionando como um detector de mentiras que impede a inteligência artificial de ignorar os artigos científicos para inventar respostas baseadas apenas em seu treinamento prévio

In [142]:
# 1. Preparação dos dados para validação
# Usamos a resposta gerada pelo Mistral e os contextos que o BGE-Large encontrou
query_text = pergunta 
answer_text = resposta
# Unimos os textos dos chunks recuperados para formar a base de comparação
contexto_consolidado = " ".join([res['text'] for res in results])

# 2. Gerar Embeddings para comparação
# Usamos o mesmo modelo BGE-Large (que já está carregado) para garantir a consistência
print("Calculando vetores de fidelidade...")
ans_emb = model.encode([answer_text])
ctx_emb = model.encode([contexto_consolidado])

# 3. Calcular Similaridade de Cosseno (Fidelidade Semântica)
# Mede o quanto a resposta "flutua" dentro do espaço semântico dos artigos
fidelidade_score = cosine_similarity(ans_emb, ctx_emb)[0][0]

# 4. Exibição da Análise de Qualidade
print("\n" + "═"*50)
print(f"🔬 RELATÓRIO DE VERIFICAÇÃO CIENTÍFICA")
print("-" * 50)
print(f"📊 Score de Fidelidade: {fidelidade_score:.4f}")

# Diagnóstico refinado para modelos de alta performance como o Mistral
if fidelidade_score > 0.82:
    status = "✅ EXCELENTE: Resposta totalmente ancorada nos artigos."
elif fidelidade_score > 0.65:
    status = "⚠️ ATENÇÃO: A resposta é relevante, mas pode conter inferências externas ao dataset."
else:
    status = "❌ ALERTA: Possível alucinação ou resposta baseada apenas no conhecimento prévio do modelo."

print(f"Status: {status}")
print("═"*50)

# Opcional: Mostrar a distância entre a pergunta original e a resposta
q_emb = model.encode([query_text])
relevancia_pergunta = cosine_similarity(ans_emb, q_emb)[0][0]
print(f"🎯 Relevância da Resposta para a Pergunta: {relevancia_pergunta:.4f}")

Calculando vetores de fidelidade...

══════════════════════════════════════════════════
🔬 RELATÓRIO DE VERIFICAÇÃO CIENTÍFICA
--------------------------------------------------
📊 Score de Fidelidade: 0.8322
Status: ✅ EXCELENTE: Resposta totalmente ancorada nos artigos.
══════════════════════════════════════════════════
🎯 Relevância da Resposta para a Pergunta: 0.7145


11.2 - metrica de alucinação negativa

a métrica de Alucinação Negativa, ou Gap Analysis, é projetada para identificar se o modelo está de fato extraindo informações úteis dos textos ou se está apenas realizando um "enrolation" técnico ao parafrasear a sua pergunta.

In [143]:
# 1. Preparação dos dados para comparação
# Usamos a resposta gerada e a pergunta feita para medir a sobreposição
ans_emb = model.encode([resposta])
query_emb = model.encode([pergunta])

# 2. Cálculo da Similaridade entre Resposta e Pergunta
# Se for muito alto, o modelo pode estar apenas "enrolando" sem trazer dados dos PDFs
sim_pergunta = cosine_similarity(ans_emb, query_emb)[0][0]

# 3. Cálculo do GAP de Alucinação / Agregação de Valor
# Comparamos o quanto a resposta se aproxima dos artigos (fidelidade_score) 
# versus o quanto ela se aproxima da pergunta (sim_pergunta)
hallucination_gap = fidelidade_score - sim_pergunta

print("\n" + "═"*50)
print(f"📊 MÉTRICA DE AGREGAÇÃO TÉCNICA (Gap Analysis)")
print("-" * 50)
print(f"📈 Fidelidade (Resposta-Contexto): {fidelidade_score:.4f}")
print(f"🔄 Repetição (Resposta-Pergunta):  {sim_pergunta:.4f}")
print("-" * 30)
print(f"🎯 GAP DE CONHECIMENTO: {hallucination_gap:.4f}")

# 4. Diagnóstico do Desempenho do RAG
if hallucination_gap > 0.10:
    status = "✅ SEGURO: O modelo extraiu e sintetizou informações novas dos artigos."
elif hallucination_gap > 0.0:
    status = "🟡 NEUTRO: O modelo seguiu a pergunta, mas a contribuição dos artigos foi moderada."
elif hallucination_gap > -0.10:
    status = "⚠️ ALERTA: A resposta está muito presa ao texto da pergunta (Parafraseamento)."
else:
    status = "❌ CRÍTICO: Possível alucinação ou resposta genérica ignorando os fatos dos PDFs."

print(f"\nStatus: {status}")
print("═"*50)


══════════════════════════════════════════════════
📊 MÉTRICA DE AGREGAÇÃO TÉCNICA (Gap Analysis)
--------------------------------------------------
📈 Fidelidade (Resposta-Contexto): 0.8322
🔄 Repetição (Resposta-Pergunta):  0.7145
------------------------------
🎯 GAP DE CONHECIMENTO: 0.1177

Status: ✅ SEGURO: O modelo extraiu e sintetizou informações novas dos artigos.
══════════════════════════════════════════════════


11.3 - métrica de densidade de informação

a Densidade de Informação foca na qualidade estrutural e na sofisticação do texto produzido, sendo fundamental para evitar que o modelo entre em ciclos de repetição ou utilize uma linguagem excessivamente genérica e simplista.

In [144]:
# 1. Tokenização e Limpeza
# Transformamos a resposta em uma lista de palavras, removendo pontuação
palavras_limpas = re.findall(r'\w+', resposta.lower())
vocabulario_unico = set(palavras_limpas)

# 2. Cálculo da Densidade (Diversidade Lexical)
# Mede a riqueza do vocabulário gerado pelo Mistral
total_palavras = len(palavras_limpas)
total_unicas = len(vocabulario_unico)
densidade = total_unicas / total_palavras if total_palavras > 0 else 0

print("\n" + "═"*50)
print(f"📊 MÉTRICA DE RIQUEZA LEXICAL")
print("-" * 50)
print(f"Total de palavras na resposta: {total_palavras}")
print(f"Vocabulário único:             {total_unicas}")
print("-" * 30)
print(f"Índice de Densidade:           {densidade:.4f}")

# 3. Diagnóstico de Qualidade Textual
# Em textos científicos, densidade alta indica uso de termos precisos (sem "encher linguiça")
if densidade > 0.65:
    status = "✅ EXCELENTE: Texto denso, técnico e sem repetições desnecessárias."
elif densidade > 0.45:
    status = "🟡 NORMAL: Fluidez adequada para uma explicação científica."
else:
    status = "⚠️ REPETITIVO: O modelo pode estar 'preso' em um loop. Considere subir o 'repetition_penalty'."

print(f"\nStatus: {status}")
print("═"*50)


══════════════════════════════════════════════════
📊 MÉTRICA DE RIQUEZA LEXICAL
--------------------------------------------------
Total de palavras na resposta: 248
Vocabulário único:             130
------------------------------
Índice de Densidade:           0.5242

Status: 🟡 NORMAL: Fluidez adequada para uma explicação científica.
══════════════════════════════════════════════════


11.4 - métrica de precisão de recuperação(analise sobre o BGE-large)

In [119]:
# 1. Ajuste: Extrair apenas os textos da lista de dicionários 'results'
# Isso garante que o modelo receba apenas strings, não metadados.
contextos_texto = [res['text'] for res in results]

# 2. Obter o embedding da pergunta original
query_embedding = model.encode([pergunta])

# 3. Obter embeddings dos contextos recuperados para validação
# Usamos o BGE que já está na memória (ou CPU)
context_embeddings = model.encode(contextos_texto)

# 4. Cálculo das métricas de busca (Similaridade de Cosseno)
similaridades = cosine_similarity(query_embedding, context_embeddings)[0]

mean_similarity = np.mean(similaridades)
max_similarity = np.max(similaridades)

# 5. Exibição da Performance do Retriever
print("\n" + "═"*50)
print(f"📊 PERFORMANCE DO RETRIEVER (BGE-LARGE)")
print("-" * 50)
print(f"Similaridade Máxima (Top 1):  {max_similarity:.4f}")
print(f"Similaridade Média (Top {len(similaridades)}):   {mean_similarity:.4f}")
print("-" * 30)

# 6. Diagnóstico de Qualidade da Recuperação
# Com o Mistral, se o Max for baixo, a chance de alucinação sobe muito.
if max_similarity > 0.75:
    status = "✅ ALTA RELEVÂNCIA: O BGE encontrou trechos muito específicos nos artigos."
elif max_similarity > 0.55:
    status = "🟡 RELEVÂNCIA MÉDIA: O conteúdo é correlato, mas pode ser genérico."
else:
    status = "❌ BAIXA RELEVÂNCIA: O banco de 1000 PDFs pode não conter a resposta exata."

print(f"Status: {status}")
print("═"*50)


══════════════════════════════════════════════════
📊 PERFORMANCE DO RETRIEVER (BGE-LARGE)
--------------------------------------------------
Similaridade Máxima (Top 1):  0.8045
Similaridade Média (Top 5):   0.7864
------------------------------
Status: ✅ ALTA RELEVÂNCIA: O BGE encontrou trechos muito específicos nos artigos.
══════════════════════════════════════════════════
